# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902033


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/39 [00:00<?, ?it/s]

Rendering models:   5%|▌         | 2/39 [00:05<01:43,  2.80s/it]

Rendering models:   8%|▊         | 3/39 [00:05<01:13,  2.04s/it]

Rendering models:  10%|█         | 4/39 [00:06<00:52,  1.50s/it]

Rendering models:  13%|█▎        | 5/39 [00:06<00:37,  1.09s/it]

...

Rendering models:  92%|█████████▏| 36/39 [00:13<00:00,  5.49it/s]

Rendering models:  95%|█████████▍| 37/39 [00:13<00:00,  5.50it/s]

Rendering models:  97%|█████████▋| 38/39 [00:13<00:00,  5.16it/s]

Rendering models: 100%|██████████| 39/39 [00:14<00:00,  4.56it/s]

equidad1                              0.043914
bobhail                               0.001553
welshgirlie                           0.044524
kayleebug2017                         0.005430
JessieDawn                            0.007696
ZigZagoon999                          0.002465
mistely                               0.002388
jvillavargas                          0.001479
not-logged-in-f3a7f56133994eacba59    0.004386
Thedunkmasta                          0.005183
Thedunkmasta                          0.004200
Thedunkmasta                          0.004749
Thedunkmasta                          0.004566
Thedunkmasta                          0.004549
Thedunkmasta                          0.004547
Thedunkmasta                          0.004486
Thedunkmasta                          0.004693
Thedunkmasta                          0.004544
sasa7                                 0.004006
not-logged-in-ab1612bc18cd7aab85db    0.002107
Alexander00                           0.003018
dxjdxjhhhh   

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())